In [ ]:
import torch
import pandas as pd
from ipywidgets import interact
import numpy as np
from matplotlib import pyplot as plt
import time

# PyTorch Basics Notebook

This notebook is a transcription of the FastAI course's "How does a neural net really work?" notebook that introduces users to `autograd` and `Tensor`.  To best get a feel for the lowest level of PyTorch after working with Lightning for some time, this is a quick trip back to the fundamentals.

Like the FastAI course, we'll start with fitting a quadratic function to noisy data using gradient descent and compare it to other optimization approaches.

In [ ]:
def quadratic_factory(a: float, b: float, c: float) -> callable:
    """Build a function that will return values defined by the quadratic function
    parametrized by a, b, and c

    Args:
        a (float): The squared coefficient
        b (float): The linear coefficient
        c (float): The constant value

    Returns:
        callable: A function that returns 'y' values for the quadratic function 
            parametrized by a, b, and c 
    """

    def quadratic(x: float | np.ndarray) -> float | np.ndarray:
        """The quadratic function"""
        return a * x ** 2 + b * x + c
    
    return quadratic

def noisy_samples(quadratic: callable, sigma: float) -> callable:
    """Wrap a defined quadratic function with a noise generator
    to generate training samples

    Args:
        quadratic (callable): The defined quadratic function, see above
        sigma (float): The variance of the gaussian noise added to every sample

    Returns:
        callable: A function that returns noisy samples of specified shape
    """
    def noisy_quad(x: float | np.ndarray):
        return quadratic(x) + np.random.randn(*x.shape) * sigma
    return noisy_quad

In [ ]:

# Define a function and plot it
a = 4
b = 3
c = -1
quad_func = quadratic_factory(a, b, c)
sampler = noisy_samples(quadratic=quad_func, sigma=5)

x = np.linspace(-5, 5, 50)
y = quad_func(x)

# Generate some samples between [-5, 5]
x_sampled = (np.random.rand(50) - .5) * 10
y_sampled = sampler(x_sampled)

plt.plot(x, y, label='F(x)')
plt.plot(x_sampled, y_sampled, 'r.', label='Sampled F(x)')
plt.title(f'f(x) = ${a}x^2 + {b}x + {c}$')
plt.ylabel('$f(x)$')
plt.xlabel('$x$')
plt.legend()
plt.show()

# Quadratic Regression

This can be solved closed-form using quadratric regression

In [ ]:
def quadratic_regression(x: np.ndarray, y: np.ndarray) -> tuple[np.ndarray, float]:
    start = time.time()
    matrix_x = np.vstack((x**2, x, np.ones_like(x))).T
    matrix_x_inv = np.dot(np.linalg.inv(np.dot(matrix_x.T, matrix_x)), matrix_x.T)
    best_fit = np.dot(matrix_x_inv, y)
    total_time = time.time() - start
    print(f'Elapsed: {total_time:.4f} seconds')
    return best_fit, total_time


In [ ]:
a, b, c = np.round(np.random.randn(3), 2)
n_samples = 250
print(f'f(x) = {a}x^2 + {b}x + {c}')
quadratic = quadratic_factory(a, b, c)
sampler = noisy_samples(quadratic, sigma=1)

x = (np.random.rand(n_samples) - .5) * 10
x.sort()
y = sampler(x)
best_fit, total_time = quadratic_regression(x, y)
fit_quadratic = quadratic_factory(*best_fit)
print(f'f`(x) = {best_fit[0]}x^2 + {best_fit[1]}x + {best_fit[2]}')

# Compute MSE
mse = np.mean((fit_quadratic(x) - y) ** 2)
print(f'Mean Squared Error: {mse}')

plt.plot(x, quadratic(x), label='F(x)')
plt.plot(x, y, 'r.', label='Sampled F(x)')
plt.plot(x, fit_quadratic(x), 'g', label='Best Fit F`(x)')
plt.title(f'f(x) = ${a}x^2 + {b}x + {c}$')
plt.ylabel('$y$')
plt.xlabel('$x$')
plt.legend()
plt.show()